# Novelty analysis
In this notebook, we'll look at the trade-offs between novelty and accuracy in semantic matching.

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
os.chdir('../../')

In [30]:
import logging
logging.getLogger().setLevel(logging.INFO)

import takco
import pandas as pd

config = takco.Config('resources/config-dbpedia.toml')
tables = takco.TableSet._load('output/t2d-v2-baseline/1-link/*')
scored_tables = tables.score('t2d-v2', keycol_only=True, config=config)

takco.preview(scored_tables)

INFO:root:Loading data from resources/t2d_fix.csv
INFO:root:Loaded 514 annotated tables


In [38]:
report = scored_tables.make_report()
scores = report['scores']
pd.DataFrame.from_dict(scores, orient='index')

INFO:root:Collected 26106 gold and 57137 pred for task entities
INFO:root:Collected 671 gold and 704 pred for task properties
INFO:root:Collected 237 gold and 584 pred for task classes


,precision,recall,f1-score,support,predictions
entities,0.298091,0.652417,0.409212,26106,57137
properties,0.211648,0.222057,0.216727,671,704
classes,0.361301,0.890295,0.514007,237,584
